In [20]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s
# from connector.mosaicio import MosaicioConnector

In [21]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [22]:
data_ind_U  = pd.read_csv(r"/data/Synthetic_data_Clustered_individual_U.csv")
#uncomment to run for org
# data_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv")

ind_data_thresholds_U = pd.read_csv(r"/data/threshold_individual_U.csv")
#uncomment to run for org
# org_data_thresholds_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_organization_U.csv")

weight_table_ind_U = pd.read_csv(r"/data/weight_individual_U.csv")
#uncomment to run for org
# weight_table_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_organization_U.csv")


In [23]:
ind_data_thresholds_U

,P002001031,P002001032,P002001033,P002001015,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001026,P002001016,P002001018,P002001019,P002001021,P002001023,P002001024,P002001025,P002001027,P002001030,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003007,P002003008,P003001002,P002001028,P002001029,OCCUPATION_BUSINESS_CLEAN,Cluster_labels
0,"(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1190476190476...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.15, 1.0, 1, 0...","(1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1076923076923...","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1414141414141...","(1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1204819277108...","(2, 3, 3, 4, 5, 6, 6, 7, 8, 9, 1.2923076923076...","(1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 1.1666666666666...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1060606060606...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 1.234375, 1.0, ...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(-74, -74, -74, -74, -74, -74, -74, -74, -74, ...","(23710, 23710, 23710, 23710, 23710, 23710, 237...","(26837, 26837, 26837, 26837, 26837, 26837, 268...","(-80, -80, -80, -80, -80, -80, -80, -80, -80, ...","(64351, 64351, 64351, 64351, 64351, 64351, 643...","(None, None, None, None, None, None, None, Non...","(-22, -22, -22, -22, -22, -22, -22, -22, -22, ...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1034482758620...","(None, None, None, None, None, None, None, Non...",Government Sector,0
1,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111...","(1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 1.0526315789473...","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.125, 1.0, 1, ...","(2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 1.571428571428...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1333333333333...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None,

In [24]:
# data_ind_U['PEER_GROUP_ID'] = data_ind_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)
ind_data_thresholds_U['PEER_GROUP_ID'] = ind_data_thresholds_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)
weight_table_ind_U['PEER_GROUP_ID'] = weight_table_ind_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)



In [36]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [37]:
ind_data_thresholds_U

,P002001031,P002001032,P002001033,P002001015,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001026,P002001016,P002001018,P002001019,P002001021,P002001023,P002001024,P002001025,P002001027,P002001030,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003007,P002003008,P003001002,P002001028,P002001029,OCCUPATION_BUSINESS_CLEAN,Cluster_labels,PEER_GROUP_ID
0,"(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1190476190476...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.15, 1.0, 1, 0...","(1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1076923076923...","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1414141414141...","(1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1204819277108...","(2, 3, 3, 4, 5, 6, 6, 7, 8, 9, 1.2923076923076...","(1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 1.1666666666666...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1060606060606...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 1.234375, 1.0, ...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(-74, -74, -74, -74, -74, -74, -74, -74, -74, ...","(23710, 23710, 23710, 23710, 23710, 23710, 237...","(26837, 26837, 26837, 26837, 26837, 26837, 268...","(-80, -80, -80, -80, -80, -80, -80, -80, -80, ...","(64351, 64351, 64351, 64351, 64351, 64351, 643...","(None, None, None, None, None, None, None, Non...","(-22, -22, -22, -22, -22, -22, -22, -22, -22, ...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1034482758620...","(None, None, None, None, None, None, None, Non...",Government Sector,0,Government Sector_0
1,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111...","(1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 1.0526315789473...","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.125, 1.0, 1, ...","(2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 1.571428571428...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1333333333333...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None,

In [38]:
# Changed Code


# tmp = cursor.execute("SELECT * FROM fincore_analytics.RISK_PERSON_SUMMARY_STG ;").fetchone()
count = 1
# Populating the threshold values in the threshold table
for ind,row in ind_data_thresholds_U.iterrows():
    if (count%10000 == 0):
        print(count," Done")
    count+=1
    for key in row.keys():
        
        if key.startswith('P0'): #only considering columns which start with P
            
            data = literal_eval(row[key])
#             print(row['PEER_GROUP_ID'],key)
            print("data = ",data)

            if all(elem is None for elem in data) == True:
                print("The rule_no",key,"don't have any value for peer_group",row['PEER_GROUP_ID'])
                conn.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (PEER_GROUP_ID,RULE_NO)values(?,?)""",(row['PEER_GROUP_ID'],key))
                print("end")

            else:   
#                 print("data = ",data)
# #                 data = literal_eval(row[key])
                conn.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,MEAN,MEDIAN,MODE,STANDARD_DEVIATION,OUTLIER,PEER_GROUP_ID,RULE_NO)values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",(data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],data[11],data[12],data[13],data[14],row['PEER_GROUP_ID'],key))

    
#Uncomment to run for org
# print("_________")
# count=1
# for ind,row in org_data_thresholds_U.iterrows():
#     if (count%10000 == 0):
#         print(count," Done")
#     count+=1
# #     print(row['PEER_GROUP_ID'])
#     for key in row.keys():
#         if key.startswith('O0'):
#             data = row[key].replace("inf","2e307")
#             data = literal_eval(data)
# #             print(key)
#             cursor.execute("""insert into fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,
#             THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,PEER_GROUP_ID,RULE_NO)
#             values(?,?,?,?,?,?,?,?,?,?,?,?)""",(data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],row['PEER_GROUP_ID'],key))
conn.commit()
# conn.close()

data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001031 don't have any value for peer_group Government Sector_0
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001032 don't have any value for peer_group Government Sector_0
end
data =  (2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0.5936328831862332, 7.0)
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.119047619047619, 1.0, 1, 0.497727260961116, 5.25)
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 1.0)
data =  (1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.15, 1.0, 1, 0.476969600708473, 5.25)
data =  (1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1076923076923078, 1.0, 1, 0.3561795970089301, 4.0)
data =  (1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1414141414141414, 1.0, 1, 0.47162091030015163, 5.5)
data =  (1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1204819277108433, 1.0, 1, 0.39263062016386324, 4.0)
data =  (2, 3, 3, 4, 5, 6, 6, 7, 8, 9, 1.29230

end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001031 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001032 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001015 don't have any value for peer_group OTHERS_CHANGED_1
end
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.25, 1.0, 1, 0.4330127018922193, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1,

end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001028 don't have any value for peer_group OTHERS_CHANGED_5
end
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001029 don't have any value for peer_group OTHERS_CHANGED_5
end
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 1.0)
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001032 don't have any value for peer_group Others_0
end
data =  (1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 1.0285714285714285, 1.0, 1, 0.1665986255670086, 2.5)
data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P002001015 don't have any value for peer_group Others_0
end
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 1.0)
data =  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 1.0)
data =  (1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 1.0909090909090908, 1.0, 1, 0.287479

In [39]:
peers = pd.read_sql_query("Select distinct peer_group_id from fincore_db.fincore_analytics.risk_person_summary_stg with (nolock) ",conn)

In [40]:
peers = peers['peer_group_id'].to_list()

In [41]:
rule_no = pd.read_sql_query("select rule_no from fincore_db.fincore_analytics.risk_lkp_rules with (nolock) where is_active_flag = 1 and RISK_SCORE_STATIC_FLAG = 0 and RULE_NO like 'P%' and risk_type <> 'Auto High Rules '",conn)

In [42]:
rule_no = rule_no['rule_no'].to_list()

In [43]:
data_ind_U.drop_duplicates(inplace = True)

In [48]:

for peer_group_id in peers:
    for rules in rule_no: 
#         print(peer_group_id,rules)
        a = data_ind_U[data_ind_U['PEER_GROUP_ID'] == peer_group_id][rules]
        a = a.dropna()
        a = a.astype('int32')
        if (len(a) == 0):
            th = ['Null','Null']
        else:
            th = []
#             print(min(a),max(a),np.mean(a),np.median(a),statistics.mode(a))
#             th[0] = min(a)
#             th[1] = max(a)
#             th[2] = np.mean(a)
#             th[3] = np.median(a)
#             th[4] = statistics.mode(a)
            th.append(min(a))
            th.append(max(a))




        cursor.execute("""update fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD set min = {} , max = {} ,update_date = getdate() where peer_group_id ='{}' and rule_no = '{}' """.format(th[0],th[1],peer_group_id,rules))
        conn.commit()
        
conn.close()



In [49]:
# # changed Code
# # iterate data_ind_U table and for each rule record its rule_no, rule_value and compare it with the risk_entity_threshold and
# # update its threshold flag and calc its risk score and append it in the risk_person table.
# #UPDATING RISK_PERSON_ANALYTICAL_RULES TABLE
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# count = 1
# threshold_table =  pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_LKP_ENTITY_THRESHOLD]",conn)
# # threshold_table.fill
# cols = ['PERSON_MASTER_ID','PEER_GROUP_ID']
# rule_df = pd.read_sql_query("SELECT * FROM [HDM_FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] where IS_ACTIVE_FLAG = 1 and RISK_SCORE_STATIC_FLAG = 0 and RISK_TYPE <> 'Auto High Rules' and PERSON_TYPE = 'Individual'", conn)
# reqd_cols_per = rule_df['RULE_NO'].to_list()
# cols.extend(reqd_cols_per)
# print(cols)
# data_ind_U_mod = data_ind_U[cols]

# external_list = []
# for indI,rowI in data_ind_U_mod.iterrows():
# #     print(rowI['PERSON_MASTER_ID'])
#     if count%10000 == 0:
#         print(indI,rowI['PERSON_MASTER_ID']," Done")    
#         conn.commit()
        
# #     if count == 100:
# #         break
#     count+=1
    
#     for indT,rowT in threshold_table.iterrows():
        
#         for key in rowI.keys():
            
#             if key.startswith('P0') & (key == rowT['RULE_NO']) & (rowI['PEER_GROUP_ID'] == rowT['PEER_GROUP_ID']):
#                 l = []
# #                 print("Here")
# #                 print(key,rowT['RULE_NO'],rowI[key],rowI['PERSON_MASTER_ID'],rowI['PEER_GROUP_ID'],rowT['PEER_GROUP_ID'])
#                 if (np.isnan(rowT['THRESHOLD1_1']) == True):
                    
#                     l = [rowI['PERSON_MASTER_ID'],key,rowI[key],0]
                
#                 elif (rowT['THRESHOLD1_10'] == 0):
#                     print("C2****************************************************")
# #                     print(key)
#                     l = [rowI['PERSON_MASTER_ID'],key,rowI[key],0]
#                     print(l)
# #                     
#                 else:
                    
#                     if (np.isnan(rowI[key]) == True):
# #                         print("Here")
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],0]
#                     elif (rowI[key] < rowT['THRESHOLD1_1']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],0]
#                     elif (rowI[key] == rowT['THRESHOLD1_1']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],1]
#                     elif (rowI[key] <= rowT['THRESHOLD1_2']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],2]
#                     elif (rowI[key] <= rowT['THRESHOLD1_3']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],3]
#                     elif (rowI[key] <= rowT['THRESHOLD1_4']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],4]
#                     elif (rowI[key] <= rowT['THRESHOLD1_5']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],5]
#                     elif (rowI[key] <= rowT['THRESHOLD1_6']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],6]
#                     elif (rowI[key] <= rowT['THRESHOLD1_7']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],7]
#                     elif (rowI[key] <= rowT['THRESHOLD1_8']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],8]
#                     elif (rowI[key] <= rowT['THRESHOLD1_9']):
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],9]

#                     else:
#                         l = [rowI['PERSON_MASTER_ID'],key,rowI[key],10]
# #                     print(l)
#                 external_list.append(l)

# #     break
# conn.commit()
# conn.close()

In [50]:
# length = len(data_ind_U_mod)
# print(length)
# z = length//10000

In [51]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [52]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [53]:
# # changed Code to run in batches

# # iterate data_ind_U table and for each rule record its rule_no, rule_value and compare it with the risk_entity_threshold and
# # update its threshold flag and calc its risk score and append it in the risk_person table.
# #UPDATING RISK_PERSON_ANALYTICAL_RULES TABLE

# threshold_table =  pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_LKP_ENTITY_THRESHOLD]",conn)
# # threshold_table.fill
# cols = ['PERSON_MASTER_ID','PEER_GROUP_ID']
# rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] where IS_ACTIVE_FLAG = 1 and RISK_SCORE_STATIC_FLAG = 0 and RISK_TYPE <> 'Auto High Rules' and PERSON_TYPE = 'Individual'", conn)
# reqd_cols_per = rule_df['RULE_NO'].to_list()
# cols.extend(reqd_cols_per)
# data_ind_U_mod = data_ind_U[cols]
# external_list = []

# data_ind_U_mod.to_csv(r"/notebooks/notebooks/data_ind_U_mod.csv",index=False)

# # source_path = r"/notebooks/notebooks/data_ind_U_mod.csv"

# # if not os.path.exists('/notebooks/notebooks/data_ind_U_mod_split'):
# #     os.makedirs('data_ind_U_mod_split')

# # for i,chunk in enumerate(pd.read_csv(source_path, chunksize=10000)):
# #     chunk.to_csv(r"/notebooks/notebooks/data_ind_U_mod_split/data_ind_U_mod{}.csv".format(i), index=False)


In [54]:
# data_ind_U_mod

In [55]:
# data_ind_U_mod.drop_duplicates(inplace = True)

In [56]:
# data_ind_U_mod.reset_index(inplace = True)

In [57]:
# data_ind_U_mod.drop(columns = ['index'],inplace = True)

In [58]:
# threshold_table[threshold_table['RULE_NO'] == 'P002001019']

In [59]:
# data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U.csv",index = False)
# ind_data_thresholds_U.to_csv(r"/data/threshold_individual_U.csv",index = False)
# weight_table_ind_U.to_csv(r"/data/weight_individual_U.csv",index = False)
# data_ind_U_mod.to_csv(r"/data/data_ind_U_mod.csv",index=False)